# 第4回講義 宿題

## 課題

今Lessonで学んだことを元に，MNISTのファッション版 (Fashion MNIST，クラス数10) を多層パーセプトロンによって分類してみましょう．

Fashion MNISTの詳細については以下のリンクを参考にしてください．

Fashion MNIST: https://github.com/zalandoresearch/fashion-mnist

### 目標値

Accuracy 85%

### ルール


- 訓練データは`x_train`， `t_train`，テストデータは`x_test`で与えられます．
- 予測ラベルは one_hot表現ではなく0~9のクラスラベル で表してください．
- **下のセルで指定されている`x_train`，`t_train`以外の学習データは使わないでください．**
- Pytorchを利用して構いません．
- ただし，**`torch.nn.Conv2d`のような高レベルのAPIは使用しないで下さい**．具体的には，`nn.Parameter`, `nn.Module`, `nn.Sequential`以外の`nn`系のAPIです．
- torchvision等で既に実装されているモデルも使用しないで下さい．

### 提出方法
- 2つのファイルを提出していただきます．
    1. テストデータ (`x_test`) に対する予測ラベルを`submission_pred.csv`として保存し，**Omnicampusの宿題タブから「第4回 ニューラルネットワークの最適化・正則化」を選択して**提出してください．
    2. それに対応するpythonのコードを`submission_code.py`として保存し，**Omnicampusの宿題タブから「第4回 ニューラルネットワークの最適化・正則化 (code)」を選択して**提出してください．pythonファイル自体の提出ではなく，「提出内容」の部分にコードをコピー&ペーストしてください．
      
- なお，採点は1で行い，2はコードの確認用として利用します（成績優秀者はコード内容を公開させていただくかもしれません）．コードの内容を変更した場合は，**1と2の両方を提出し直してください**．

### 評価方法
- 予測ラベルの`t_test`に対する精度 (Accuracy) で評価します．
- 即時採点しLeader Boardを更新します（採点スケジュールは別アナウンス）．
- 締切時の点数を最終的な評価とします．

### ドライブのマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### データの読み込み（この部分は修正しないでください）

`__len__`は，Pythonの組み込み関数len()を呼んだときに，内部で呼ばれる特殊メソッドです．

`__getitem__`は，インデックスやキーで要素を取得するときに，内部で呼ばれる特殊メソッドです．

どちらも， Datasetクラスを自作する際によく登場します．

```python
class MyList:
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]

mylist = MyList([10, 20, 30])
print(len(mylist))  # __len__が呼び出される
# 3
print(mylist[1])  # __getitem__が呼び出される
# 20
```

In [19]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import inspect

nn_except = ["Module", "Parameter", "Sequential"]
for m in inspect.getmembers(nn):
    if not m[0] in nn_except and m[0][0:2] != "__":
        delattr(nn, m[0])

seed = 1234
torch.manual_seed(seed)
np.random.seed(seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#学習データ
x_train = np.load('../data/x_train.npy')
t_train = np.load('../data/y_train.npy')

#テストデータ
x_test = np.load('../data/x_test.npy')

class train_dataset(torch.utils.data.Dataset):
    def __init__(self, x_train, t_train):
        self.x_train = x_train.reshape(-1, 784).astype('float32') / 255
        self.t_train = t_train

    def __len__(self):
        return self.x_train.shape[0]

    def __getitem__(self, idx):
        return torch.tensor(self.x_train[idx], dtype=torch.float), torch.tensor(self.t_train[idx], dtype=torch.long)

class test_dataset(torch.utils.data.Dataset):
    def __init__(self, x_test):
        self.x_test = x_test.reshape(-1, 784).astype('float32') / 255

    def __len__(self):
        return self.x_test.shape[0]

    def __getitem__(self, idx):
        return torch.tensor(self.x_test[idx], dtype=torch.float)

trainval_data = train_dataset(x_train, t_train)
test_data = test_dataset(x_test)

### 多層パーセプトロンの実装

In [20]:
batch_size = 32

val_size = 10000
train_size = len(trainval_data) - val_size

train_data, val_data = torch.utils.data.random_split(trainval_data, [train_size, val_size])

dataloader_train = torch.utils.data.DataLoader(
    train_data,
    batch_size=batch_size,
    shuffle=True
)

dataloader_valid = torch.utils.data.DataLoader(
    val_data,
    batch_size=batch_size,
    shuffle=True
)

dataloader_test = torch.utils.data.DataLoader(
    test_data,
    batch_size=batch_size,
    shuffle=False
)

In [21]:
def relu(x):
    return torch.where(x > 0, x, torch.zeros_like(x))

def softmax(x):
    x_max = x.max(dim=1, keepdim=True)[0]
    exp_x = torch.exp(x - x_max)
    return exp_x / exp_x.sum(dim=1, keepdim=True)


class Dense(nn.Module):
    def __init__(self, in_dim, out_dim):
        super().__init__()
        # 重みとバイアスをパラメータとして定義
        self.W = nn.Parameter(torch.randn(in_dim, out_dim) * 0.01)
        self.b = nn.Parameter(torch.zeros(out_dim))

    def forward(self, x):
        # x: (batch, in_dim)
        # 出力: (batch, out_dim)
        return x.matmul(self.W) + self.b

class MLP(nn.Module):
    def __init__(self, in_dim, hid_dim, out_dim):
        super().__init__()
        self.l1 = Dense(in_dim, hid_dim)
        self.l2 = Dense(hid_dim, out_dim)

    def forward(self, x):
        h = relu(self.l1(x))
        y = softmax(self.l2(h))
        return y
in_dim = 784
hid_dim = 200
out_dim = 10
lr = 0.001
n_epochs = 10

mlp = MLP(in_dim, hid_dim, out_dim).to(device)

optimizer = optim.SGD(mlp.parameters(), lr=lr)

AttributeError: module 'torch.nn' has no attribute 'attention'

In [ ]:
for epoch in range(n_epochs):
    losses_train = []
    losses_valid = []
    train_num = 0
    train_true_num = 0
    valid_num = 0
    valid_true_num = 0

    mlp.train()  # 訓練時には勾配を計算するtrainモードにする
    for x, t in dataloader_train:
        # WRITE ME

        losses_train.append(loss.tolist())

        acc = torch.where(t - pred.to("cpu") == 0, torch.ones_like(t), torch.zeros_like(t))
        train_num += acc.size()[0]
        train_true_num += acc.sum().item()

    mlp.eval()  # 評価時には勾配を計算しないevalモードにする
    for x, t in dataloader_valid:
        # WRITE ME

        losses_valid.append(loss.tolist())

        acc = torch.where(t - pred.to("cpu") == 0, torch.ones_like(t), torch.zeros_like(t))
        valid_num += acc.size()[0]
        valid_true_num += acc.sum().item()

    print('EPOCH: {}, Train [Loss: {:.3f}, Accuracy: {:.3f}], Valid [Loss: {:.3f}, Accuracy: {:.3f}]'.format(
        epoch,
        np.mean(losses_train),
        train_true_num/train_num,
        np.mean(losses_valid),
        valid_true_num/valid_num
    ))

In [ ]:
mlp.eval()

t_pred = []
for x in dataloader_test:

    x = x.to(device)

    # 順伝播
    y = mlp.forward(x)

    # モデルの出力を予測値のスカラーに変換
    pred = y.argmax(1).tolist()

    t_pred.extend(pred)

submission = pd.Series(t_pred, name='label')
submission.to_csv('drive/MyDrive/Colab Notebooks/DLBasics2025_colab/Lecture04/submission_pred.csv', header=True, index_label='id')